## Categorise expenses

In [416]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import json
import os.path as path


In [425]:
#This is the file used to categorise the expenses based on previous years ones.
#If there is an available one that hasn't been processed yet please upload it to the folder
#Classification based only on the description
#First read one if it is present

if path.exists("dictionary_training.json") :   
    with open('dictionary_training.json') as f: 
        data = f.read() 
    dic = json.loads(data) 
else:
    dic = {}
    
if path.exists('expenses_train.csv'):
    expenses = pd.read_csv('expenses_train.csv', skiprows=1, skip_blank_lines=True)
else:
    if len(dic) == 0:
        print('No previous dictionary and no data to train a new one are available. The program will continue with no categorisation')
    
    
for k,v in expenses.groupby('Description').groups.items():
    dic[k] = expenses.iloc[v.values[0]].notna().tolist()  

with open('dictionary_training.json', 'w') as d:
    json.dump(dic, d)    

In [426]:
#Income only
account = pd.read_csv('account.csv', skiprows=4, skip_blank_lines=True)
account = account.dropna(axis=0, how='all')
account = account.drop('Balance', axis=1)
account = pd.concat([account,pd.DataFrame(columns=expenses.columns[5:])])
#account['Date'] = pd.to_datetime(account['Date'])

income = pd.DataFrame(columns=account.columns)
income['Date'] = pd.to_datetime(income['Date'])
for i in account.index.tolist():    
    if pd.notnull(account.loc[i]['Paid in']):
        income = income.append(account.loc[i])
        account = account.drop(labels=i)
income['Sales income'] = income['Paid in']

income[income.columns] = income[income.columns].replace({'£': ''}, regex=True)
income.to_csv('sales.csv')

In [422]:
account = pd.read_csv('account.csv', skiprows=4, skip_blank_lines=True)
account = account.dropna(axis=0, how='all')
account = account.drop('Balance', axis=1)
account = pd.concat([account,pd.DataFrame(columns=expenses.columns[5:])])

review_expenses = pd.DataFrame(columns=account.columns)
for i in range(account.shape[0]):    
    if account.iloc[i]['Description'] in dic:
        account.iloc[i,5:] = account.iloc[i,3]
        for j in range(len(account.columns)):
            account.iloc[i,j] = account.iloc[i,j] * dic[account.iloc[i]['Description']][j]
    else:
        review_expenses = review_expenses.append(account.iloc[i])
account.to_csv('categorised_expenses.csv')

total = pd.concat([income,account])
total['Date'] = pd.to_datetime(total['Date'])
total = total.sort_values(by="Date")

total[total.columns] = total[total.columns].replace({'£': ''}, regex=True)
review_expenses[review_expenses.columns] = review_expenses[review_expenses.columns].replace({'£': ''}, regex=True)
total.to_csv('total.csv')
review_expenses.to_csv('review_expenses.csv')

After manually categorising the review_expenses.csv file please upload it as "expenses_train.csv" and run the whole code once more to include new categories into the dictionary.